In [4]:
from bs4 import BeautifulSoup
import pandas as pd
import requests
import json
import pprint
pp = pprint.PrettyPrinter(indent=4)

In [5]:
with open('ToposTextThucycides.html') as ifh:
    html = ''.join(ifh.readlines())

In [6]:
# print(html[:300])

In [7]:
soup = BeautifulSoup(html, 'html.parser')

In [8]:

p_list = soup.findAll("p")

# for x in p_list[1:6]:
#     print(x, '\n')

In [14]:
import pandas as pd
import re


book_line_pat = re.compile('(?P<book>[0-9]+)\.(?P<line>[0-9]+)')
id_from_link_pat = re.compile('place/(?P<id>.+)$')


names = []
longs = []
lats = []
links = []
books = []
lines = []
ids = []


for p in p_list:
    places = p.findAll("a", class_ = "place")
    if places != []:
        string_with_book_and_line = p.b.text
        book = book_line_pat.search(string_with_book_and_line).group('book')
        line = book_line_pat.search(string_with_book_and_line).group('line')
#         print(string_with_book_and_line)
        for place in places:
            name = place.text
            long = place.get("long")
            lat = place.get("lat")
            link = place.get("about")
            placeID = id_from_link_pat.search(link).group('id')
            names.append(name)
            longs.append(long)
            lats.append(lat)
            links.append(link)
            ids.append(placeID)
            books.append(book)
            lines.append(line)
            
            

            
            
place_df = pd.DataFrame({'name': names, 'placeID': ids, 'longitude': longs, "latitude": lats, "link": links, "book": books, "line": lines})




link_counts = {}
for link in place_df['link']:
    if link in link_counts:
        link_counts[link] += 1
    else:
        link_counts[link] = 1

#print(place_df.head(10))

# for k in link_counts:
#     print(k, link_counts[k])




# for i in range(len(place_df)):
#     print(place_df['name'], i)






          name     placeID longitude latitude  \
0     Thessaly  394223RThs      22.3     39.4   
1      Boeotia  384233RBoi     23.25    38.35   
2  Peloponnese  375222RPel     22.15    37.54   
3      Arcadia  375223RArk      22.3     37.5   
4       Attica  379239RAtt      23.9     37.9   
5       Attica  379239RAtt      23.9     37.9   
6        Ionia  375275RIon      27.7       38   
7    Phthiotis  393223RPht      22.5     39.2   
8    Phthiotis  393223RPht      22.5     39.2   
9     Cyclades  372248RKyk      24.8     37.2   

                                     link book line  
0  https://topostext.org/place/394223RThs    1    2  
1  https://topostext.org/place/384233RBoi    1    2  
2  https://topostext.org/place/375222RPel    1    2  
3  https://topostext.org/place/375223RArk    1    2  
4  https://topostext.org/place/379239RAtt    1    2  
5  https://topostext.org/place/379239RAtt    1    2  
6  https://topostext.org/place/375275RIon    1    2  
7  https://topostext.org/pla

In [15]:
counts = []
link_list = place_df['link'].tolist()
for i in range(len(place_df)):
#     print(place_df['name'], i)
    counts.append(link_counts[link_list[i]])
#     print(place_df['link'], i)

In [18]:
place_df['count'] = counts

#print(place_df.head(10))


for k in link_counts:
    print(k, link_counts[k])

https://topostext.org/place/394223RThs 13
https://topostext.org/place/384233RBoi 38
https://topostext.org/place/375222RPel 156
https://topostext.org/place/375223RArk 7
https://topostext.org/place/379239RAtt 66
https://topostext.org/place/375275RIon 24
https://topostext.org/place/393223RPht 4
https://topostext.org/place/372248RKyk 2
https://topostext.org/place/374253PDel 17
https://topostext.org/place/380237PAth 379
https://topostext.org/place/400262UIli 9
https://topostext.org/place/377228PMyk 3
https://topostext.org/place/376227PArg 71
https://topostext.org/place/371224PSpa 198
https://topostext.org/place/403265LChe 7
https://topostext.org/place/385229UArn 1
https://topostext.org/place/406163RIta 24
https://topostext.org/place/376145RSic 175
https://topostext.org/place/379229PKor 95
https://topostext.org/place/377269PSam 94
https://topostext.org/place/408187WIon 9
https://topostext.org/place/374252PRhe 3
https://topostext.org/place/433054PMas 1
https://topostext.org/place/378234PAig 2

In [19]:
# This is the request for the geojson:

params = {
    'work_id': 2
}
r = requests.get('https://topostext.org/api/place/geojsonwork.php', params=params)
    

In [20]:
geoj = r.text

# print(geoj)
# pp.pprint(geoj)

with open('geoj.json', "w") as gjfh:
    gjfh.writelines(geoj)

In [21]:
# load JSON as dictionary
geoj_dict=json.loads(geoj)


# print(geoj_dict)

In [22]:
places_dict_list = geoj_dict['features']

In [23]:
placeID_dict = {}

for place in places_dict_list:
    id = place['properties']['placeID']
    if place['properties']['featureTypes'] == 'sanctuary/temple':
        featureTypes = 'temple'
    else:
        featureTypes = place['properties']['featureTypes']
    description = place['properties']['description']
    greek = place['properties']['greek2']
    placeID_dict[id] = {'featureTypes': featureTypes, 'description': description, 'greek': greek}

    
    
    
# pp.pprint(placeID_dict)

In [32]:
featureTypes = []
descriptions = []
id_list = place_df['placeID'].tolist()

for i in range(len(place_df)):
#     print(i, placeID_dict[id_list[i]])
    try:
        featureTypes.append(placeID_dict[id_list[i]]['featureTypes'])
        descriptions.append(placeID_dict[id_list[i]]['description'])
    except KeyError:
        featureTypes.append('?')
        descriptions.append('?')

# print(id_list[64])
# print(id_list[65]) #problem!
# print(id_list[66])

# print(placeID_dict[id_list[64]])
# print(placeID_dict[id_list[64]]['featureTypes'])
# print(placeID_dict[id_list[65]]) #problem!
# print(placeID_dict[id_list[65]]['featureTypes']) #problem!

# for i in range(len(featureTypes)):
#     print(i, featureTypes[i], descriptions[i])



# Several entries in the placeID_dict appear to be missing, e.g. Eteonus (383235UEte), although the entry exists in Topos Text

In [33]:
place_df['featureTypes'] = featureTypes
place_df['description'] = descriptions

In [34]:
# export pandas dataframe to csv file
place_df.to_csv(r'topos_places.csv')

In [35]:
no_duplicates_df = place_df.drop_duplicates(subset=['name'])
no_duplicates_df.to_csv(r'no_duplicates_topos_places.csv')

In [36]:
# !cat topos_places.csv

In [37]:
last_book = max([int(x) for x in place_df['book'].tolist()])
# print(last_book)

In [38]:
for i in range(last_book):
    book = i + 1
#     print(book)
    df_name = "book_" + str(book) + '.csv'
    single_book_df = place_df[place_df.book == str(book)]
    single_book_df.to_csv(df_name)
    no_duplicates_book_df = single_book_df.drop_duplicates(subset=['name'])
    no_duplicates_book_df.to_csv('no_duplicates' + df_name)
